<a href="https://colab.research.google.com/github/daryoush/minGPT/blob/master/startjulia1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
%cd /gdrive/My Drive/
! git clone https://github.com/daryoush/minGPT.git

#! git pull
#! pip install <desired-python-library>

[Errno 2] No such file or directory: 'gdrive/My Drive/'
/content/gdrive/My Drive
Cloning into 'minGPT'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 198 (delta 6), reused 14 (delta 1), pack-reused 176
Receiving objects: 100% (198/198), 1.84 MiB | 15.11 MiB/s, done.
Resolving deltas: 100% (107/107), done.


# New Section

In [2]:
### NOTE THIS WORKS AS PYTHON, IT ONLY WORKS WHEN THE RUNTIME COMPLAINS ABOUT JULIA RUNTIME, SO IT REVERTS TO PYTHON. AFTER JULIA IS INSTALLED, CHANGE TO JULIA RUNTIME

!curl -sSL "https://julialang-s3.julialang.org/bin/linux/x64/1.5/julia-1.5.0-linux-x86_64.tar.gz" -o julia.tar.gz
!tar -xzf julia.tar.gz -C /usr --strip-components 1
!rm -rf julia.tar.gz*
!julia -e 'using Pkg; pkg"add IJulia; precompile"'
!julia -e 'using Pkg; Pkg.add([ "CUDA", "Flux","Zygote","Profile", "TensorBoardLogger", "Logging",\
    "Random","InformationMeasures", "DataStructures", "Transformers","Statistics", "Dates",\
    "BSON",\
    "Images",\
    "MLDatasets",\
    "Parameters",\
    "Printf",\
    ]);' ###Pkg.precompile()'
!echo "DONE"

 Installing known registries into `~/.julia`
######################################################################## 100.0%
      Added registry `General` to `~/.julia/registries/General`
  Resolving package versions...
  Installed VersionParsing ── v1.2.0
  Installed ZeroMQ_jll ────── v4.3.2+5
  Installed SoftGlobalScope ─ v1.1.0
  Installed Conda ─────────── v1.5.0
  Installed ZMQ ───────────── v1.2.1
  Installed JSON ──────────── v0.21.1
  Installed IJulia ────────── v1.23.1
  Installed JLLWrappers ───── v1.1.3
  Installed Artifacts ─────── v1.3.0
  Installed MbedTLS_jll ───── v2.16.8+1
  Installed MbedTLS ───────── v1.0.3
  Installed Parsers ───────── v1.0.15
######################################################################## 100.0%
######################################################################## 100.0%
Updating `~/.julia/environments/v1.5/Project.toml`
  [7073ff75] + IJulia v1.23.1
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [56f22d72] + Artifacts v1.3.0
  

**NOW GO TO EDIT NOTEBOOK SETTING AND SET IT TO JULIA**
For adding new packages, it should be in python or the syntax may need to be in Julia!

In [1]:
[1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [2]:
import Pkg; Pkg.add("BenchmarkTools")

   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [12]:
using BenchmarkTools

M = rand(2048, 2048)
@benchmark M^2

BenchmarkTools.Trial: 
  memory estimate:  32.00 MiB
  allocs estimate:  2
  --------------
  minimum time:     417.434 ms (0.00% GC)
  median time:      431.325 ms (0.00% GC)
  mean time:        435.868 ms (0.73% GC)
  maximum time:     471.040 ms (0.71% GC)
  --------------
  samples:          12
  evals/sample:     1

In [13]:
if ENV["COLAB_GPU"] == "1"
    using CUDA

    M_gpu = cu(M)
    @benchmark CUDA.@sync M_gpu^2
else
    println("No GPU found.")
end

BenchmarkTools.Trial: 
  memory estimate:  416 bytes
  allocs estimate:  11
  --------------
  minimum time:     2.757 ms (0.00% GC)
  median time:      4.951 ms (0.00% GC)
  mean time:        5.025 ms (0.03% GC)
  maximum time:     9.150 ms (17.12% GC)
  --------------
  samples:          994
  evals/sample:     1

In [17]:
M_gpu = nothing
M=nothing

In [7]:
cd("/content/gdrive/MyDrive/minGPT")
pwd()

"/content/gdrive/MyDrive/minGPT"

In [2]:
using Flux: onehot
	using Transformers: Vocabulary, decode
	using Random
	using Flux: throttle, @epochs, ADAMW, ADAM, RADAM, Momentum, Optimiser, logitcrossentropy, params
	using Flux: softmax, onecold
	using DataStructures: Queue, enqueue!, dequeue!
	Random.seed!(42)


MersenneTwister(UInt32[0x0000002a], Random.DSFMT.DSFMT_state(Int32[964434469, 1073036706, 1860149520, 1073503458, 1687169063, 1073083486, -399267803, 1072983952, -909620556, 1072836235  …  -293054293, 1073002412, -1300127419, 1073642642, 1917177374, -666058738, -337596527, 1830741494, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x0000000000

In [3]:
import Pkg; Pkg.add("NNlib")
Pkg.add("Distributions")
Pkg.add("Juno")

   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [8]:
using Flux

module Datasets include("onJulia/src/datasets.jl") end
dataSetlength, vocab, sample = Datasets.openDataset("input.txt")
batch(dataLength,batchLength ) = hcat((sample(dataLength) for x in 1:batchLength)...)

datasize = length(s) = 1115394


batch (generic function with 1 method)

In [9]:
## ---
module Model include("onJulia/src/model.jl") end

lengthOfSentences=520

520

In [20]:

model=Model.minGPT(maxBlockSize=lengthOfSentences, vocabSize=length(vocab)) |> gpu

LoadError: ignored

In [11]:




## ---


# for a batch of sentences, extract 1st to one before last character as input
# and 2nd to last char of each sentence as prediction.  Shape the predictred
# values as a long one dim series of values, one hot encode them using the vocab

trainingDataPair(b)=(b[1:end-1, :], onehot(vocab, reshape(b[2:end,:], :)))
trainingData(dataLength,batchLength)= [trainingDataPair(batch(dataLength,batchLength))
		for i in 1:dataSetlength/(dataLength*batchLength)]

function loss(x,y)
	yhat=model(x)
	logitcrossentropy(reshape(yhat,size(yhat)[1], :), y)
end


function validate()
	s=collect("Wher")
	for i in 1:20
		out=model(vocab(s))
		push!(s, decode(vocab,onecold(softmax(out)[:,end])))
	end
	join(s)
end

validate (generic function with 1 method)

In [12]:
opt = RADAM()
opt = ADAM( 0.0006, (0.9, 0.95))

## ---
module Train include("onJulia/src/train.jl") end

Main.Train

In [13]:
function sampleFromModel(s="Whe", minprob=.05) 
	function explore(x, minprob)
		x[1] < minprob && return currentRes
		out=model(vocab(collect(x[2])))
		choices=sort(collect(zip(softmax(out[:,end]), decode(vocab, collect(1:size(out)[1])))), rev=true)
		options = [(x[1]*np, x[2]*nc) for (np,nc) in choices if (x[1]*np) > minprob ]
	end
	toexplore=Queue{Tuple{Float32,String}}()
	enqueue!(toexplore,(1, s))  #start with the initial string and prob 1
	results=[]
	for i in toexplore
		nxt=dequeue!(toexplore)
		res=explore(nxt, minprob)
		if isempty(res)  # go as long as you can't add any more character and satisfy the prob
			push!(results, nxt)
			continue #continue emptying the queue
		end
		for r in res
			enqueue!(toexplore, r)  ## add elements for further exploration
		end
	end
    #elements of the results are (prob, string), sort from high to low of the 
	#longest match with highest prob
	sort(results, by=x->(length(x[2]), x[1]), rev=true)
end


sampleFromModel (generic function with 3 methods)

In [14]:
sampleFromModel("Where i", 0.01)

┌ Warning: Performing scalar operations on GPU arrays: This is very slow, consider disallowing these operations with `allowscalar(false)`
└ @ GPUArrays /root/.julia/packages/GPUArrays/uaFZh/src/host/indexing.jl:43


49-element Array{Any,1}:
 (0.04530935f0, "Where iq")
 (0.038606867f0, "Where iO")
 (0.034068882f0, "Where iJ")
 (0.032712847f0, "Where iC")
 (0.027322533f0, "Where ig")
 (0.025702143f0, "Where ih")
 (0.023709433f0, "Where ir")
 (0.022544838f0, "Where iU")
 (0.021729756f0, "Where iQ")
 (0.020966258f0, "Where iH")
 (0.019722583f0, "Where i3")
 (0.01960443f0, "Where ia")
 (0.019196505f0, "Where i:")
 ⋮
 (0.013056043f0, "Where iI")
 (0.012859254f0, "Where iR")
 (0.0117791565f0, "Where if")
 (0.011755045f0, "Where i'")
 (0.011690839f0, "Where ie")
 (0.011527548f0, "Where iz")
 (0.011345717f0, "Where iP")
 (0.01132936f0, "Where iE")
 (0.01125623f0, "Where iA")
 (0.011068826f0, "Where iT")
 (0.011046158f0, "Where i;")
 (0.010519223f0, "Where i\n")

In [51]:
numberOfSentencesInBatch=10
ps = params(model)
for i in 1:10 
  d=trainingData(lengthOfSentences, numberOfSentencesInBatch) |> gpu
  Train.train!(loss,
            ps,
            d,
            opt,
            sampleFromModel )
end



ctr = 0
(ctr, training_loss) = (0, 1.2593999f0)
training_accuracy = validate() = Any[(0.070789635f0, "When you "), (0.07702288f0, "When he "), (0.05578764f0, "When the"), (0.06958025f0, "When w")]
ctr = 1
ctr = 2
ctr = 3
ctr = 4
ctr = 5
ctr = 6
ctr = 7
ctr = 8
ctr = 9
ctr = 10
(ctr, training_loss) = (10, 1.1814114f0)
training_accuracy = validate() = Any[(0.09659533f0, "When he "), (0.058231067f0, "When I "), (0.15487602f0, "Where "), (0.06883715f0, "When w")]
ctr = 11
ctr = 12
ctr = 13
ctr = 14
ctr = 15
ctr = 16
ctr = 17
ctr = 18
ctr = 19
ctr = 20
(ctr, training_loss) = (20, 1.2411944f0)
training_accuracy = validate() = Any[(0.050597377f0, "When the "), (0.10436567f0, "When he "), (0.059964698f0, "When I "), (0.15290536f0, "Where "), (0.053673558f0, "When w")]
ctr = 21
ctr = 22
ctr = 23
ctr = 24
ctr = 25
ctr = 26
ctr = 27
ctr = 28
ctr = 29
ctr = 30
(ctr, training_loss) = (30, 1.2476472f0)
training_accuracy = validate() = Any[(0.06364054f0, "When he "), (0.050115757f0, "Where's "), (0.1

In [32]:
Pkg.add("BSON")
using BSON

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [58]:
using BSON:  @save, @load
@save "inputtext_model.bson"  model

In [16]:
using Zygote
using Flux
using Transformers
using CUDA
using BSON:  @save, @load
@load "inputtext_model.bson"  model


In [17]:
sampleFromModel("I don't und", 0.001)


LoadError: ignored

In [ ]:
s

In [20]:
Pkg.add("StatsBase")
Pkg.add("Plots")

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
  Installed Ogg_jll ───────── v1.3.4+2
  Installed LAME_jll ──────── v3.100.0+3
  Installed x265_jll ──────── v3.0.0+3
  Installed PlotThemes ────── v1.0.3
  Installed RecipesPipeline ─ v0.1.13
  Installed Plots ─────────── v1.0.14
  Installed Contour ───────── v0.5.7
  Installed GeometryTypes ─── v0.8.4
  Installed Showoff ───────── v0.3.2
  Installed Grisu ─────────── v1.0.0
  Installed FreeType2_jll ─── v2.10.1+5
  Installed Opus_jll ──────── v1.3.1+3
  Installed FFMPEG_jll ────── v4.3.1+4
  Installed LibVPX_jll ────── v1.9.0+1
  Installed libfdk_aac_jll ── v0.1.6+4
  Installed PlotUtils ─────── v0.6.5
  Installed libvorbis_jll ─── v1.3.6+6
  Installed FFMPEG ────────── v0.3.0
  Installed Bzip2_jll ─────── v1.0.6+5
  Installed Measures ──────── v0.3.1
  Installed FriBidi_jll ───── v1.0.5+6
  Installed libass_

In [47]:
using StatsBase
using Plots
function getOutputResults(m, data, limit=5)
	output=[]
	ctr=0
	for (x,y) in data
		out=softmax(m(x))
		out=reshape(out,size(out)[1], :)
		push!(output, onecold(out)...)  ## get the encoded vocab using straight onecold, not onecold(vocab, ..) version

		ctr +=1
		if  ctr > limit break end
	end
	convert(Array{Int}, output)
end

d=trainingData(lengthOfSentences, numberOfSentencesInBatch)
function OutputHistogram(m)
	outChars=getOutputResults(m, d)
	fit(Histogram, outChars, 0:1:length(vocab))
end

#Try a untrained model
raw=OutputHistogram(Model.minGPT(maxBlockSize=lengthOfSentences, vocabSize=length(vocab)))

#..a. and trained model
trained=OutputHistogram(model)

plot(raw)
plot!(trained)

LoadError: ignored

In [39]:
CUDA.memory_status()


LoadError: ignored

In [19]:
model=nothing; GC.gc(true)

In [20]:
CUDA.memory_status()


Effective GPU memory usage: 8.28% (1.219 GiB/14.726 GiB)
CUDA allocator usage: 1.047 GiB
binned usage: 1.047 GiB (0 bytes allocated, 1.047 GiB cached)


In [21]:
CUDA.reclaim()

1124073472

In [22]:
CUDA.memory_status()


Effective GPU memory usage: 1.17% (175.875 MiB/14.726 GiB)
CUDA allocator usage: 0 bytes
binned usage: 0 bytes (0 bytes allocated, 0 bytes cached)
